# Build Admission-Level Structured Dataset

This notebook assembles a single-row-per-admission dataset (`hadm_id`) with multi-label ICD-10 targets and structured features from `patients`, `admissions`, `diagnoses_icd`, `procedures_icd`, `prescriptions`, and `labevents`. Chartevents are deferred in v1.


In [5]:
import os
import json
from datetime import datetime
from typing import List, Dict, Tuple

import numpy as np
import pandas as pd

DATA_DIR = "/Users/benjamindykstra/development/icd-10-coding/data"
PROCESSED_DIR = "/Users/benjamindykstra/development/icd-10-coding/data/processed"
os.makedirs(PROCESSED_DIR, exist_ok=True)

# File paths
PATIENTS_CSV = os.path.join(DATA_DIR, "patients.csv")
ADMISSIONS_CSV = os.path.join(DATA_DIR, "admissions.csv")
DIAGNOSES_CSV = os.path.join(DATA_DIR, "diagnoses_icd.csv")
PROCEDURES_CSV = os.path.join(DATA_DIR, "procedures_icd.csv")
PRESCRIPTIONS_CSV = os.path.join(DATA_DIR, "prescriptions.csv")
LABEVENTS_CSV = os.path.join(DATA_DIR, "labevents.csv")
D_ICD_DIAGNOSES_CSV = os.path.join(DATA_DIR, "d_icd_diagnoses.csv")

# Home-health ICD-10 target groups (prefix/range rules)
LABEL_GROUPS: Dict[str, List[str]] = {
    # Use prefixes; ranges handled by checking first 3 chars and comparing
    "y_diabetes": ["E08", "E09", "E10", "E11", "E13", "E12"],  # include E12 defensively
    "y_heart_failure": ["I50"],
    "y_copd": ["J44"],
    "y_pressure_ulcer": ["L89"],
    "y_aftercare": ["Z48"],
    "y_rehab": ["Z47", "Z50"],
    "y_hypertension": ["I10", "I11", "I12", "I13", "I15"],
    "y_stroke_sequelae": ["I69"],
}

# Medication class keyword heuristics (lowercased substring match)
MED_CLASS_KEYWORDS: Dict[str, List[str]] = {
    "med_insulin_flag": ["insulin"],
    "med_diuretic_flag": [
        "furosemide", "lasix", "bumetanide", "bumex", "torsemide", "demadex",
        "hydrochlorothiazide", "hctz", "chlorthalidone", "spironolactone",
        "eplerenone", "metolazone"
    ],
    "med_anticoagulant_flag": [
        "warfarin", "coumadin", "heparin", "enoxaparin", "lovenox",
        "apixaban", "eliquis", "rivaroxaban", "xarelto", "dabigatran", "pradaxa"
    ],
    "med_antibiotic_flag": [
        "amoxic", "augmentin", "penicillin", "cephal", "cef", "ceftriaxone", "zosyn",
        "piperacillin", "tazobactam", "vancomycin", "azithro", "levofloxacin",
        "ciprofloxacin", "doxycycline", "metronidazole"
    ],
    "med_pain_management_flag": [
        "morphine", "oxycodone", "hydrocodone", "fentanyl", "tramadol",
        "acetaminophen", "tylenol", "ibuprofen", "naproxen"
    ],
}

# Chunk size for large tables
CHUNK_ROWS = 2_000_000

pd.options.mode.copy_on_write = True


In [6]:
# Helper functions

def parse_timestamp(series: pd.Series) -> pd.Series:
    """Parse timestamps robustly."""
    return pd.to_datetime(series, errors="coerce")


def compute_age_at_admit(anchor_age: pd.Series,
                          anchor_year: pd.Series,
                          admittime: pd.Series) -> pd.Series:
    """Compute age at admission using MIMIC-IV anchor scheme."""
    admit_year = pd.to_datetime(admittime, errors="coerce").dt.year
    age = anchor_age + (admit_year - anchor_year)
    return age.clip(lower=0)


def icd10_prefix_match(code: str, prefixes: List[str]) -> bool:
    """Return True if ICD-10 code starts with any of the prefixes.
    Codes may contain dots; comparison uses the alphanumeric prefix.
    """
    if not isinstance(code, str) or not code:
        return False
    norm = code.replace(".", "").upper()
    for p in prefixes:
        if norm.startswith(p):
            return True
    return False


# Charlson Comorbidity Index (simplified ICD-10 mapping)
# Mapping adapted to common 3-character blocks; conservative inclusion.
CHARLSON_BLOCKS: Dict[str, int] = {
    # Myocardial infarction
    "I21": 1, "I22": 1, "I252": 1,
    # Congestive heart failure
    "I50": 1, "I110": 1, "I130": 1, "I132": 1,
    # Peripheral vascular disease
    "I70": 1, "I71": 1, "I72": 1, "I73": 1, "I77": 1, "I79": 1, "K55": 1, "Z958": 1, "Z959": 1,
    # Cerebrovascular disease
    "I60": 1, "I61": 1, "I62": 1, "I63": 1, "I64": 1, "I65": 1, "I66": 1, "I69": 1,
    # Dementia
    "F00": 1, "F01": 1, "F02": 1, "F03": 1, "G30": 1,
    # COPD
    "J40": 1, "J41": 1, "J42": 1, "J43": 1, "J44": 1, "J45": 1, "J46": 1, "J47": 1,
    # Connective tissue disease
    "M05": 1, "M06": 1, "M32": 1, "M33": 1, "M34": 1, "M35": 1, "M36": 1,
    # Peptic ulcer disease
    "K25": 1, "K26": 1, "K27": 1, "K28": 1,
    # Mild liver disease
    "K70": 1, "K71": 1, "K72": 1, "K73": 1, "K74": 1, "K76": 1, "B18": 1,
    # Diabetes (uncomplicated)
    "E10": 1, "E11": 1, "E12": 1, "E13": 1, "E14": 1,
    # Hemiplegia/paraplegia
    "G81": 2, "G82": 2, "G83": 2,
    # Renal disease
    "N18": 2, "N19": 2, "N052": 2, "N042": 2, "Z992": 2,
    # Any malignancy
    "C00": 2, "C01": 2, "C02": 2, "C03": 2, "C04": 2, "C05": 2, "C06": 2, "C07": 2,
    "C08": 2, "C09": 2, "C10": 2, "C11": 2, "C12": 2, "C13": 2, "C14": 2, "C15": 2,
    "C16": 2, "C17": 2, "C18": 2, "C19": 2, "C20": 2, "C21": 2, "C22": 2, "C23": 2,
    "C24": 2, "C25": 2, "C26": 2, "C34": 2, "C50": 2, "C67": 2, "C71": 2, "C73": 2,
    # Moderate/severe liver disease
    "K704": 3, "K711": 3, "K721": 3, "K729": 3, "K765": 3,
    # Metastatic solid tumor
    "C77": 6, "C78": 6, "C79": 6, "C80": 6,
    # AIDS
    "B20": 6, "B21": 6, "B22": 6, "B24": 6,
}


def compute_charlson_icd10(icd10_codes: List[str]) -> int:
    """Compute a simplified Charlson Comorbidity Index from ICD-10 codes.
    We sum the maximum weight per block present.
    """
    if not icd10_codes:
        return 0
    blocks_present: Dict[str, int] = {}
    for code in icd10_codes:
        if not isinstance(code, str):
            continue
        norm = code.replace(".", "").upper()
        # Try 4-char then 3-char blocks for specificity
        candidates = {norm[:4], norm[:3]}
        for cand in candidates:
            if cand in CHARLSON_BLOCKS:
                blocks_present[cand] = max(blocks_present.get(cand, 0), CHARLSON_BLOCKS[cand])
    return int(sum(blocks_present.values()))


def summarize_text_presence(series: pd.Series) -> float:
    """Return fraction of non-null entries in a series (missingness complement)."""
    if len(series) == 0:
        return 0.0
    return float(series.notna().mean())


In [7]:
# Load base tables (patients, admissions)
patients = pd.read_csv(PATIENTS_CSV)
admissions = pd.read_csv(ADMISSIONS_CSV)

# Parse times
admissions["admittime"] = parse_timestamp(admissions["admittime"]) 
admissions["dischtime"] = parse_timestamp(admissions["dischtime"])

# Base frame per hadm_id
base = admissions[[
    "subject_id", "hadm_id", "admittime", "dischtime", "admission_type",
    "admission_location", "discharge_location", "insurance", "marital_status", "race"
]].copy()

# Attach demographics
patients_dem = patients[["subject_id", "gender", "anchor_age", "anchor_year"]].copy()
base = base.merge(patients_dem, on="subject_id", how="left")

# Compute age and LOS
base["age_at_admit"] = compute_age_at_admit(
    base["anchor_age"], base["anchor_year"], base["admittime"]
)
base["length_of_stay_days"] = (
    (base["dischtime"] - base["admittime"]).dt.total_seconds() / (3600 * 24)
)

# Prior admissions count per subject strictly before current admittime
admit_times = base[["subject_id", "hadm_id", "admittime"]].sort_values(["subject_id", "admittime"]) 
prior_counts = (
    admit_times
    .groupby("subject_id")
    .apply(lambda df: df.assign(prior_admissions_count=np.arange(len(df))))
    .reset_index(drop=True)[["hadm_id", "prior_admissions_count"]]
)
base = base.merge(prior_counts, on="hadm_id", how="left")

# Ensure types
base["prior_admissions_count"] = base["prior_admissions_count"].fillna(0).astype(int)

print(f"Base admissions: {len(base):,}")
base.head()


Base admissions: 546,028


/var/folders/bq/0x2c77dd791b4v2h4vx5j1_40000gn/T/ipykernel_22924/1523924520.py:32: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda df: df.assign(prior_admissions_count=np.arange(len(df))))


,subject_id,hadm_id,admittime,dischtime,admission_type,admission_location,discharge_location,insurance,marital_status,race,gender,anchor_age,anchor_year,age_at_admit,length_of_stay_days,prior_admissions_count
0,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,URGENT,TRANSFER FROM HOSPITAL,HOME,Medicaid,WIDOWED,WHITE,F,52,2180,52,0.786111,0
1,10000032,22841357,2180-06-26 18:27:00,2180-06-27 18:49:00,EW EMER.,EMERGENCY ROOM,HOME,Medicaid,WIDOWED,WHITE,F,52,2180,52,1.015278,1
2,10000032,25742920,2180-08-05 23:44:00,2180-08-07 17:50:00,EW EMER.,EMERGENCY ROOM,HOSPICE,Medicaid,WIDOWED,WHITE,F,52,2180,52,1.754167,3
3,10000032,29079034,2180-07-23 12:35:00,2180-07-25 17:55:00,EW EMER.,EMERGENCY ROOM,HOME,Medicaid,WIDOWED,WHITE,F,52,2180,52,2.222222,2
4,10000068,25022803,2160-03-03 23:16:00,2160-03-04 06:26:00,EU OBSERVATION,EMERGENCY ROOM,NaN,NaN,SINGLE,WHITE,F,19,2160,19,0.298611,0


In [8]:
# Build labels and diagnosis-derived features (chunked)
label_cols = list(LABEL_GROUPS.keys())

hadm_to_labels: Dict[int, Dict[str, int]] = {}
hadm_to_codes: Dict[int, List[str]] = {}

usecols = ["hadm_id", "icd_code", "icd_version", "seq_num"]
reader = pd.read_csv(DIAGNOSES_CSV, usecols=usecols, chunksize=CHUNK_ROWS)
for chunk in reader:
    icd10 = chunk[chunk["icd_version"] == 10].copy()
    if icd10.empty:
        continue
    # Normalize codes
    icd10["icd_code"] = icd10["icd_code"].astype(str)
    # Aggregate per hadm_id
    for hadm_id, df in icd10.groupby("hadm_id"):
        codes = df["icd_code"].tolist()
        # Init if needed
        if hadm_id not in hadm_to_labels:
            hadm_to_labels[hadm_id] = {k: 0 for k in label_cols}
            hadm_to_codes[hadm_id] = []
        hadm_to_codes[hadm_id].extend(codes)
        for label, prefixes in LABEL_GROUPS.items():
            if any(icd10_prefix_match(c, prefixes) for c in codes):
                hadm_to_labels[hadm_id][label] = 1

# Convert to DataFrame
labels_df = pd.DataFrame.from_dict(hadm_to_labels, orient="index").reset_index().rename(columns={"index": "hadm_id"})
labels_df = labels_df.astype({c: "int8" for c in label_cols})

# Diagnosis counts and Charlson index
codes_df = (
    pd.Series(hadm_to_codes).reset_index().rename(columns={"index": "hadm_id", 0: "codes"})
)

def diag_features(row) -> Tuple[int, int]:
    codes = row["codes"] or []
    num_total = len(codes)
    # secondary dx not trackable w/o reliable seq_num retention; default 0 for v1
    num_secondary = max(0, num_total - 1)
    return num_total, num_secondary

codes_df["num_diagnoses_total"], codes_df["num_secondary_dx"] = zip(*codes_df.apply(diag_features, axis=1))
codes_df["charlson_index"] = codes_df["codes"].apply(compute_charlson_icd10)

# Merge into base
base = base.merge(labels_df, on="hadm_id", how="left")
base = base.merge(codes_df[["hadm_id", "num_diagnoses_total", "num_secondary_dx", "charlson_index"]], on="hadm_id", how="left")

# Fill NA for admissions without ICD-10 codes
for c in label_cols:
    base[c] = base[c].fillna(0).astype("int8")
base["num_diagnoses_total"] = base["num_diagnoses_total"].fillna(0).astype(int)
base["num_secondary_dx"] = base["num_secondary_dx"].fillna(0).astype(int)
base["charlson_index"] = base["charlson_index"].fillna(0).astype(int)

print("Labels built. Positive counts:")
print(base[label_cols].sum().sort_values(ascending=False))
base.head()


Labels built. Positive counts:
y_hypertension       136439
y_diabetes            66173
y_heart_failure       42990
y_copd                21655
y_stroke_sequelae      6525
y_pressure_ulcer       4225
y_aftercare             305
y_rehab                 163
dtype: int64


,subject_id,hadm_id,admittime,dischtime,admission_type,admission_location,discharge_location,insurance,marital_status,race,...,y_heart_failure,y_copd,y_pressure_ulcer,y_aftercare,y_rehab,y_hypertension,y_stroke_sequelae,num_diagnoses_total,num_secondary_dx,charlson_index
0,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,URGENT,TRANSFER FROM HOSPITAL,HOME,Medicaid,WIDOWED,WHITE,...,0,0,0,0,0,0,0,0,0,0
1,10000032,22841357,2180-06-26 18:27:00,2180-06-27 18:49:00,EW EMER.,EMERGENCY ROOM,HOME,Medicaid,WIDOWED,WHITE,...,0,0,0,0,0,0,0,0,0,0
2,10000032,25742920,2180-08-05 23:44:00,2180-08-07 17:50:00,EW EMER.,EMERGENCY ROOM,HOSPICE,Medicaid,WIDOWED,WHITE,...,0,0,0,0,0,0,0,0,0,0
3,10000032,29079034,2180-07-23 12:35:00,2180-07-25 17:55:00,EW EMER.,EMERGENCY ROOM,HOME,Medicaid,WIDOWED,WHITE,...,0,0,0,0,0,0,0,0,0,0
4,10000068,25022803,2160-03-03 23:16:00,2160-03-04 06:26:00,EU OBSERVATION,EMERGENCY ROOM,NaN,NaN,SINGLE,WHITE,...,0,0,0,0,0,0,0,0,0,0


In [10]:
# Procedures features (chunked)
proc_usecols = ["hadm_id", "icd_code", "icd_version"]
proc_reader = pd.read_csv(PROCEDURES_CSV, usecols=proc_usecols, chunksize=CHUNK_ROWS)

proc_counts: Dict[int, int] = {}
proc_surgical_flag: Dict[int, int] = {}

for chunk in proc_reader:
    icd10 = chunk[chunk["icd_version"] == 10].copy()
    if icd10.empty:
        continue
    icd10["icd_code"] = icd10["icd_code"].astype(str).str.replace(".", "", regex=False).str.upper()
    grp = icd10.groupby("hadm_id")
    # Count
    counts = grp.size()
    for hadm_id, cnt in counts.items():
        proc_counts[hadm_id] = proc_counts.get(hadm_id, 0) + int(cnt)
    # Surgical flag heuristic: ICD-10-PCS first section character (0–9) indicates procedure section.
    # We'll flag as surgical if first char in {'0','1','2','3','4','5','7','8','9'} (broad).
    first_chars = grp["icd_code"].apply(lambda s: set(c[0] for c in s))
    for hadm_id, chars in first_chars.items():
        is_surg = any(ch in set(list("012345789")) for ch in chars)
        proc_surgical_flag[hadm_id] = int(max(proc_surgical_flag.get(hadm_id, 0), int(is_surg)))

proc_df = pd.DataFrame({
    "hadm_id": list(set(list(proc_counts.keys()) + list(proc_surgical_flag.keys()))),
})
proc_df["num_procedures_total"] = proc_df["hadm_id"].map(proc_counts).fillna(0).astype(int)
proc_df["had_surgical_flag"] = proc_df["hadm_id"].map(proc_surgical_flag).fillna(0).astype("int8")

base = base.merge(proc_df, on="hadm_id", how="left")
base["num_procedures_total"] = base["num_procedures_total"].fillna(0).astype(int)
base["had_surgical_flag"] = base["had_surgical_flag"].fillna(0).astype("int8")

print("Procedures features done.")
base.head()


Procedures features done.


,subject_id,hadm_id,admittime,dischtime,admission_type,admission_location,discharge_location,insurance,marital_status,race,...,y_pressure_ulcer,y_aftercare,y_rehab,y_hypertension,y_stroke_sequelae,num_diagnoses_total,num_secondary_dx,charlson_index,num_procedures_total,had_surgical_flag
0,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,URGENT,TRANSFER FROM HOSPITAL,HOME,Medicaid,WIDOWED,WHITE,...,0,0,0,0,0,0,0,0,0,0
1,10000032,22841357,2180-06-26 18:27:00,2180-06-27 18:49:00,EW EMER.,EMERGENCY ROOM,HOME,Medicaid,WIDOWED,WHITE,...,0,0,0,0,0,0,0,0,0,0
2,10000032,25742920,2180-08-05 23:44:00,2180-08-07 17:50:00,EW EMER.,EMERGENCY ROOM,HOSPICE,Medicaid,WIDOWED,WHITE,...,0,0,0,0,0,0,0,0,0,0
3,10000032,29079034,2180-07-23 12:35:00,2180-07-25 17:55:00,EW EMER.,EMERGENCY ROOM,HOME,Medicaid,WIDOWED,WHITE,...,0,0,0,0,0,0,0,0,0,0
4,10000068,25022803,2160-03-03 23:16:00,2160-03-04 06:26:00,EU OBSERVATION,EMERGENCY ROOM,NaN,NaN,SINGLE,WHITE,...,0,0,0,0,0,0,0,0,0,0


In [11]:
# Medications features (chunked)
med_usecols = ["hadm_id", "drug", "starttime", "stoptime"]
med_reader = pd.read_csv(PRESCRIPTIONS_CSV, usecols=med_usecols, chunksize=CHUNK_ROWS)

med_counts: Dict[int, int] = {}
med_class_flags: Dict[int, Dict[str, int]] = {}

for chunk in med_reader:
    if chunk.empty:
        continue
    # Normalize drug names
    drugs = chunk[["hadm_id", "drug"]].dropna()
    drugs["drug"] = drugs["drug"].astype(str).str.lower()
    grp = drugs.groupby("hadm_id")["drug"].apply(list)
    for hadm_id, drug_list in grp.items():
        med_counts[hadm_id] = med_counts.get(hadm_id, 0) + len(drug_list)
        # init
        if hadm_id not in med_class_flags:
            med_class_flags[hadm_id] = {k: 0 for k in MED_CLASS_KEYWORDS.keys()}
        # class flags
        for cls, keywords in MED_CLASS_KEYWORDS.items():
            if med_class_flags[hadm_id][cls] == 1:
                continue
            if any(any(kw in d for kw in keywords) for d in drug_list):
                med_class_flags[hadm_id][cls] = 1

med_df = pd.DataFrame({"hadm_id": list(set(med_counts.keys()) | set(med_class_flags.keys()))})
med_df["medication_count"] = med_df["hadm_id"].map(med_counts).fillna(0).astype(int)
med_df["polypharmacy_flag"] = (med_df["medication_count"] >= 5).astype("int8")
for cls in MED_CLASS_KEYWORDS.keys():
    med_df[cls] = med_df["hadm_id"].map(lambda h: med_class_flags.get(h, {}).get(cls, 0)).fillna(0).astype("int8")

base = base.merge(med_df, on="hadm_id", how="left")
base["medication_count"] = base["medication_count"].fillna(0).astype(int)
base["polypharmacy_flag"] = base["polypharmacy_flag"].fillna(0).astype("int8")
for cls in MED_CLASS_KEYWORDS.keys():
    base[cls] = base[cls].fillna(0).astype("int8")

print("Medications features done.")
base.head()


Medications features done.


,subject_id,hadm_id,admittime,dischtime,admission_type,admission_location,discharge_location,insurance,marital_status,race,...,charlson_index,num_procedures_total,had_surgical_flag,medication_count,polypharmacy_flag,med_insulin_flag,med_diuretic_flag,med_anticoagulant_flag,med_antibiotic_flag,med_pain_management_flag
0,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,URGENT,TRANSFER FROM HOSPITAL,HOME,Medicaid,WIDOWED,WHITE,...,0,0,0,14,1,0,1,1,0,1
1,10000032,22841357,2180-06-26 18:27:00,2180-06-27 18:49:00,EW EMER.,EMERGENCY ROOM,HOME,Medicaid,WIDOWED,WHITE,...,0,0,0,15,1,0,1,1,0,1
2,10000032,25742920,2180-08-05 23:44:00,2180-08-07 17:50:00,EW EMER.,EMERGENCY ROOM,HOSPICE,Medicaid,WIDOWED,WHITE,...,0,0,0,28,1,1,1,1,0,1
3,10000032,29079034,2180-07-23 12:35:00,2180-07-25 17:55:00,EW EMER.,EMERGENCY ROOM,HOME,Medicaid,WIDOWED,WHITE,...,0,0,0,24,1,0,0,1,0,1
4,10000068,25022803,2160-03-03 23:16:00,2160-03-04 06:26:00,EU OBSERVATION,EMERGENCY ROOM,NaN,NaN,SINGLE,WHITE,...,0,0,0,0,0,0,0,0,0,0


In [12]:
# Lab features (chunked; restrict to admission window)
lab_usecols = ["hadm_id", "itemid", "charttime", "value", "valuenum", "flag"]

# We'll attempt to identify HbA1c and BNP by common textual patterns if available via value/label, but in this CSV we have itemid only.
# Without d_labitems, we compute generic features: counts, unique items, fraction abnormal.

lab_counts: Dict[int, int] = {}
lab_unique_items: Dict[int, set] = {}
lab_abnormal_counts: Dict[int, int] = {}

# Build a quick lookup for admission windows
hadm_windows = base.set_index("hadm_id")[ ["admittime", "dischtime"] ]

reader = pd.read_csv(LABEVENTS_CSV, usecols=lab_usecols, chunksize=CHUNK_ROWS)
for chunk in reader:
    chunk["charttime"] = parse_timestamp(chunk["charttime"]) 
    # Inner join on hadm_id present in base to reduce volume
    chunk = chunk.merge(hadm_windows, left_on="hadm_id", right_index=True, how="inner")
    # Clip to admission window
    in_window = (chunk["charttime"] >= chunk["admittime"]) & (chunk["charttime"] <= chunk["dischtime"]) 
    chunk = chunk.loc[in_window]
    if chunk.empty:
        continue
    # Counts
    for hadm_id, df in chunk.groupby("hadm_id"):
        lab_counts[hadm_id] = lab_counts.get(hadm_id, 0) + len(df)
        # unique items
        items = set(df["itemid"].dropna().astype(int).tolist())
        if hadm_id not in lab_unique_items:
            lab_unique_items[hadm_id] = set()
        lab_unique_items[hadm_id].update(items)
        # abnormal counts (if flag present)
        if "flag" in df.columns:
            abn = df["flag"].astype(str).str.upper().str.contains("ABN|HIGH|LOW|CRIT|ALERT")
            lab_abnormal_counts[hadm_id] = lab_abnormal_counts.get(hadm_id, 0) + int(abn.sum())

lab_df = pd.DataFrame({"hadm_id": list(set(list(lab_counts.keys()) + list(lab_unique_items.keys())))})
lab_df["num_lab_events"] = lab_df["hadm_id"].map(lab_counts).fillna(0).astype(int)
lab_df["num_unique_lab_items"] = lab_df["hadm_id"].map(lambda h: len(lab_unique_items.get(h, set()))).astype(int)
# fraction abnormal
abn_series = lab_df["hadm_id"].map(lambda h: lab_abnormal_counts.get(h, 0)).fillna(0).astype(int)
lab_df["fraction_abnormal"] = (abn_series / lab_df["num_lab_events"].replace(0, np.nan)).fillna(0.0)
lab_df["labs_missingness_rate"] = (lab_df["num_lab_events"] > 0).astype(float)

base = base.merge(lab_df, on="hadm_id", how="left")
base["num_lab_events"] = base["num_lab_events"].fillna(0).astype(int)
base["num_unique_lab_items"] = base["num_unique_lab_items"].fillna(0).astype(int)
base["fraction_abnormal"] = base["fraction_abnormal"].fillna(0.0).astype(float)
base["labs_missingness_rate"] = base["labs_missingness_rate"].fillna(0.0).astype(float)

print("Lab features done.")
base.head()


Lab features done.


,subject_id,hadm_id,admittime,dischtime,admission_type,admission_location,discharge_location,insurance,marital_status,race,...,polypharmacy_flag,med_insulin_flag,med_diuretic_flag,med_anticoagulant_flag,med_antibiotic_flag,med_pain_management_flag,num_lab_events,num_unique_lab_items,fraction_abnormal,labs_missingness_rate
0,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,URGENT,TRANSFER FROM HOSPITAL,HOME,Medicaid,WIDOWED,WHITE,...,1,0,1,1,0,1,57,57,0.333333,1.0
1,10000032,22841357,2180-06-26 18:27:00,2180-06-27 18:49:00,EW EMER.,EMERGENCY ROOM,HOME,Medicaid,WIDOWED,WHITE,...,1,0,1,1,0,1,46,46,0.413043,1.0
2,10000032,25742920,2180-08-05 23:44:00,2180-08-07 17:50:00,EW EMER.,EMERGENCY ROOM,HOSPICE,Medicaid,WIDOWED,WHITE,...,1,1,1,1,0,1,69,28,0.579710,1.0
3,10000032,29079034,2180-07-23 12:35:00,2180-07-25 17:55:00,EW EMER.,EMERGENCY ROOM,HOME,Medicaid,WIDOWED,WHITE,...,1,0,0,1,0,1,68,30,0.573529,1.0
4,10000068,25022803,2160-03-03 23:16:00,2160-03-04 06:26:00,EU OBSERVATION,EMERGENCY ROOM,NaN,NaN,SINGLE,WHITE,...,0,0,0,0,0,0,0,0,0.000000,0.0


In [13]:
base.columns

Index(['subject_id', 'hadm_id', 'admittime', 'dischtime', 'admission_type',
       'admission_location', 'discharge_location', 'insurance',
       'marital_status', 'race', 'gender', 'anchor_age', 'anchor_year',
       'age_at_admit', 'length_of_stay_days', 'prior_admissions_count',
       'y_diabetes', 'y_heart_failure', 'y_copd', 'y_pressure_ulcer',
       'y_aftercare', 'y_rehab', 'y_hypertension', 'y_stroke_sequelae',
       'num_diagnoses_total', 'num_secondary_dx', 'charlson_index',
       'num_procedures_total', 'had_surgical_flag', 'medication_count',
       'polypharmacy_flag', 'med_insulin_flag', 'med_diuretic_flag',
       'med_anticoagulant_flag', 'med_antibiotic_flag',
       'med_pain_management_flag', 'num_lab_events', 'num_unique_lab_items',
       'fraction_abnormal', 'labs_missingness_rate'],
      dtype='object')

In [14]:
LABEL_GROUPS.keys()

dict_keys(['y_diabetes', 'y_heart_failure', 'y_copd', 'y_pressure_ulcer', 'y_aftercare', 'y_rehab', 'y_hypertension', 'y_stroke_sequelae'])

In [17]:
# Assemble, validate, and save
# Select and order columns roughly per schema
feature_cols = [
    # demographics/admission
    "subject_id", "hadm_id", "admittime", "dischtime", "age_at_admit", "gender",
    "admission_type", "discharge_location", "length_of_stay_days", "prior_admissions_count",
    # diagnoses features
    "num_diagnoses_total", "num_secondary_dx", "charlson_index",
    # procedures
    "num_procedures_total", "had_surgical_flag",
    # meds
    "medication_count", "polypharmacy_flag",
] + list(MED_CLASS_KEYWORDS.keys()) + [
    # labs
    "num_lab_events", "num_unique_lab_items", "fraction_abnormal", "labs_missingness_rate",
] + list(LABEL_GROUPS.keys())

# Drop helpers not needed
final_df = base[feature_cols].copy()

# Validations
assert final_df["hadm_id"].is_unique, "Duplicate hadm_id rows found"
assert final_df["admittime"].notna().all(), "Missing admittime"
# assert (final_df["length_of_stay_days"] >= 0).all(), "Negative LOS found"

# Save parquet
parquet_path = os.path.join(PROCESSED_DIR, "structured_dataset.parquet")
final_df.to_parquet(parquet_path, index=False)

# Save schema JSON
schema = {
    "columns": [
        {"name": c, "dtype": str(final_df[c].dtype)} for c in final_df.columns
    ],
    "n_rows": int(len(final_df)),
    "generated_at": datetime.utcnow().isoformat() + "Z",
}
with open(os.path.join(PROCESSED_DIR, "structured_dataset.schema.json"), "w") as f:
    json.dump(schema, f, indent=2)

print(f"Saved dataset to: {parquet_path}")
final_df.head()


Saved dataset to: /Users/benjamindykstra/development/icd-10-coding/data/processed/structured_dataset.parquet


,subject_id,hadm_id,admittime,dischtime,age_at_admit,gender,admission_type,discharge_location,length_of_stay_days,prior_admissions_count,...,fraction_abnormal,labs_missingness_rate,y_diabetes,y_heart_failure,y_copd,y_pressure_ulcer,y_aftercare,y_rehab,y_hypertension,y_stroke_sequelae
0,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,52,F,URGENT,HOME,0.786111,0,...,0.333333,1.0,0,0,0,0,0,0,0,0
1,10000032,22841357,2180-06-26 18:27:00,2180-06-27 18:49:00,52,F,EW EMER.,HOME,1.015278,1,...,0.413043,1.0,0,0,0,0,0,0,0,0
2,10000032,25742920,2180-08-05 23:44:00,2180-08-07 17:50:00,52,F,EW EMER.,HOSPICE,1.754167,3,...,0.579710,1.0,0,0,0,0,0,0,0,0
3,10000032,29079034,2180-07-23 12:35:00,2180-07-25 17:55:00,52,F,EW EMER.,HOME,2.222222,2,...,0.573529,1.0,0,0,0,0,0,0,0,0
4,10000068,25022803,2160-03-03 23:16:00,2160-03-04 06:26:00,19,F,EU OBSERVATION,NaN,0.298611,0,...,0.000000,0.0,0,0,0,0,0,0,0,0


In [16]:
final_df[final_df['length_of_stay_days'] < 0]

,subject_id,hadm_id,admittime,dischtime,age_at_admit,gender,admission_type,discharge_location,length_of_stay_days,prior_admissions_count,...,fraction_abnormal,labs_missingness_rate,y_diabetes,y_heart_failure,y_copd,y_pressure_ulcer,y_aftercare,y_rehab,y_hypertension,y_stroke_sequelae
1913,10035271,26463092,2165-08-13 14:00:00,2165-08-13 00:00:00,59,M,EW EMER.,DIED,-0.583333,0,...,0.0,0.0,0,0,0,0,0,0,0,0
2677,10049642,26714941,2177-02-20 03:15:00,2177-02-20 02:30:00,70,F,EW EMER.,DIED,-0.031250,0,...,0.0,0.0,0,0,0,0,0,0,0,0
3961,10077373,23874669,2187-12-18 19:14:00,2187-12-18 12:22:00,79,F,DIRECT OBSERVATION,NaN,-0.286111,0,...,0.0,0.0,0,0,0,0,0,0,0,0
6919,10136283,29334858,2163-03-01 20:43:00,2163-03-01 02:32:00,22,F,EU OBSERVATION,NaN,-0.757639,0,...,0.0,0.0,0,0,0,0,0,0,0,0
14081,10271174,26331353,2117-01-09 02:57:00,2117-01-09 02:19:00,76,M,EW EMER.,DIED,-0.026389,0,...,0.0,0.0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
527166,19649539,20159343,2117-05-27 22:04:00,2117-05-27 21:16:00,20,F,EU OBSERVATION,NaN,-0.033333,0,...,0.0,0.0,0,0,0,0,0,0,0,0
539630,19887023,24942188,2128-11-12 09:54:00,2128-11-12 00:00:00,91,M,EW EMER.,DIED,-0.412500,0,...,0.0,0.0,0,0,0,0,0,0,0,0
540596,19904685,24196469,2171-12-10 14:49:00,2171-12-10 00:00:00,49,M,EW EMER.,DIED,-0.617361,0,...,0.0,0.0,0,0,0,0,0,0,0,0
541279,19914761,22348782,2145-07-10 18:28:00,2145-07-10 00:01:00,90,M,EW EMER.,DIED,-0.768750,3,...,0.0,0.0,0,0,0,0,0,0,0,0


In [20]:
final_df.columns

Index(['subject_id', 'hadm_id', 'admittime', 'dischtime', 'age_at_admit',
       'gender', 'admission_type', 'discharge_location', 'length_of_stay_days',
       'prior_admissions_count', 'num_diagnoses_total', 'num_secondary_dx',
       'charlson_index', 'num_procedures_total', 'had_surgical_flag',
       'medication_count', 'polypharmacy_flag', 'med_insulin_flag',
       'med_diuretic_flag', 'med_anticoagulant_flag', 'med_antibiotic_flag',
       'med_pain_management_flag', 'num_lab_events', 'num_unique_lab_items',
       'fraction_abnormal', 'labs_missingness_rate', 'y_diabetes',
       'y_heart_failure', 'y_copd', 'y_pressure_ulcer', 'y_aftercare',
       'y_rehab', 'y_hypertension', 'y_stroke_sequelae'],
      dtype='object')

In [18]:
# Create subject-level splits
from sklearn.model_selection import train_test_split

# Build a subject-level table with label prevalence to stratify roughly
subj_labels = final_df.groupby("subject_id")[list(LABEL_GROUPS.keys())].max().reset_index()
# Create a coarse stratify key by binning the sum of labels
subj_labels["label_sum"] = subj_labels[list(LABEL_GROUPS.keys())].sum(axis=1)
subj_labels["strat_key"] = pd.cut(subj_labels["label_sum"], bins=[-1,0,1,2,10], labels=["0","1","2","3+"]).astype(str)

subj_train, subj_temp = train_test_split(
    subj_labels, test_size=0.30, random_state=42, stratify=subj_labels["strat_key"]
)
subj_val, subj_test = train_test_split(
    subj_temp, test_size=0.50, random_state=42, stratify=subj_temp["strat_key"]
)

# Map to hadm rows
train_df = final_df[final_df["subject_id"].isin(subj_train["subject_id"])].copy()
val_df = final_df[final_df["subject_id"].isin(subj_val["subject_id"])].copy()
test_df = final_df[final_df["subject_id"].isin(subj_test["subject_id"])].copy()

train_path = os.path.join(PROCESSED_DIR, "structured_dataset.train.parquet")
val_path = os.path.join(PROCESSED_DIR, "structured_dataset.val.parquet")
test_path = os.path.join(PROCESSED_DIR, "structured_dataset.test.parquet")
train_df.to_parquet(train_path, index=False)
val_df.to_parquet(val_path, index=False)
test_df.to_parquet(test_path, index=False)

print({"train": len(train_df), "val": len(val_df), "test": len(test_df)})


{'train': 382866, 'val': 81895, 'test': 81267}


In [19]:
base['discharge_location'].value_counts()

discharge_location
HOME                            194204
HOME HEALTH CARE                 99305
SKILLED NURSING FACILITY         52657
REHAB                            13845
DIED                             11721
CHRONIC/LONG TERM ACUTE CARE      8125
HOSPICE                           5397
AGAINST ADVICE                    3393
PSYCH FACILITY                    2965
ACUTE HOSPITAL                    2334
OTHER FACILITY                    1592
ASSISTED LIVING                    622
HEALTHCARE FACILITY                 50
Name: count, dtype: int64

# I want to build a dataset that I can predict ICD-10 codes from from referral style data


Demographics (patients.csv and admissions.csv):

Age, gender
Admission type (emergency, elective, etc.)


Clinical Context:

Primary diagnosis on admission
Recent procedures (from procedures_icd.csv)
Discharge disposition (home with services, SNF, etc.)


Medications:

Number of medications (prescriptions.csv)
Presence of specific drug classes (insulin, diuretics, anticoagulants, etc.)
Poly-pharmacy flags


Lab Values (if available in labevents.csv):

HbA1c (diabetes marker)
BNP (heart failure marker)
Key vitals at discharge


#### not sure where I can get this info
Comorbidity Indicators:

Charlson comorbidity index
Number of previous admissions

In [1]:
import pandas as pd

patients = pd.read_csv('./data/patients.csv')

In [2]:
patients.shape

(364627, 6)

In [3]:
patients.head()

,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod
0,10000032,F,52,2180,2014 - 2016,2180-09-09
1,10000048,F,23,2126,2008 - 2010,NaN
2,10000058,F,33,2168,2020 - 2022,NaN
3,10000068,F,19,2160,2008 - 2010,NaN
4,10000084,M,72,2160,2017 - 2019,2161-02-13


In [4]:
diags = pd.read_csv('./data/diagnoses_icd.csv')
diags.shape
diags.head()

# procedures = pd.read_csv('./data/procedures_icd.csv')
# procedures.shape
# procedures.head()

,subject_id,hadm_id,seq_num,icd_code,icd_version
0,10000032,22595853,1,5723,9
1,10000032,22595853,2,78959,9
2,10000032,22595853,3,5715,9
3,10000032,22595853,4,07070,9
4,10000032,22595853,5,496,9
